# IBM Applied Data Science Capstone Course by Coursera  

### Week 3 Part 1

### Import Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


In [2]:
pip install lxml

You are using pip version 19.0.3, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install BeautifulSoup4

You are using pip version 19.0.3, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install html5lib

You are using pip version 19.0.3, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


### Scrap Data from Wikipedia using BeautifulSoup

In [5]:
# send the GET request
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [6]:
# create three lists to store table data
postalCodeList = []
boroughList = []
neighborhoodList = []

In [7]:
# find the table
soup.find('table').find_all('tr')

# find all the rows of the table
soup.find('table').find_all('tr')

# for each row of the table, find all the table data
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')

In [8]:
# append the data into the respective lists
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text.rstrip('\n'))

In [9]:
# create a new DataFrame from the three lists
toronto_df = pd.DataFrame({"PostalCode": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})

toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Drop cells with a borough that is "Not assigned"

In [10]:
toronto_df_dropna = toronto_df[toronto_df.Borough != "Not assigned"].reset_index(drop=True)
toronto_df_dropna.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


### Group neighborhoods in the same borough

In [11]:
# group neighborhoods in the same borough
toronto_df_grouped = toronto_df_dropna.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### For Neighborhood="Not assigned", make the value the same as Borough

In [12]:
# for Neighborhood="Not assigned", make the value the same as Borough
for index, row in toronto_df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [13]:
# print the number of rows of the cleaned dataframe
toronto_df_grouped.shape

(103, 3)

## Part 2

In [14]:
# load the coordinates from the csv file on Coursera
coordinates = pd.read_csv("/Users/prateek/Desktop/IBM/Geospatial_Coordinates.csv") #Change pathname
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
# rename the column "PostalCode"
coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Merge two tables to get coordinates

In [16]:
# merge two table on the column "PostalCode"
toronto_df_new = toronto_df_grouped.merge(coordinates, on="PostalCode", how="left")
toronto_df_new.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [17]:
# create a new test dataframe
column_names = ["PostalCode", "Borough", "Neighborhood", "Latitude", "Longitude"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(toronto_df_new[toronto_df_new["PostalCode"]==postcode], ignore_index=True)
    
test_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,"Maryvale, Wexford",43.750072,-79.295849
7,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420


## Part 3

We will choose "Downtown Toronto" as the borough of interest and then cluster the neighborhoods around it including those in the boroughs with 'Toronto' in their names, e.g. East Toronto, Central Toronto.

In [18]:
toronto_data = test_df[test_df['Borough'].str.contains('Toronto')].reset_index(drop = True)
toronto_data

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M4M,East Toronto,Studio District,43.659526,-79.340923
2,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420
3,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636


In [19]:
address = 'Downtown Toronto, TO'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Downtown Toronto are 43.6541737, -79.3808116451341.


In [21]:
print(*zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']))

(43.6579524, -79.3873826, 'Central Bay Street') (43.6595255, -79.340923, 'Studio District') (43.6289467, -79.3944199, 'CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara') (43.6542599, -79.3606359, 'Harbourfront, Regent Park')


In [28]:
# lets create a map of Toronto city using the Latitude and Longitude information  
Toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# Using the location information from geospat file, we add markers on the map for each Neighborhood

for lat, lng, borough, Neighborhood in zip(test_df['Latitude'], test_df['Longitude'], test_df['Borough'], test_df['Neighborhood']):
    label = '{}, {}'.format(Neighborhood, borough) # This will label the markers by Neighborhood, borough name
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        ).add_to(Toronto_map)  
    
Toronto_map

In [29]:
# Lets create a map of Downtown Toronto and its surrounding Neighborhoods
map_dwtor = folium.Map(location=[latitude, longitude], zoom_start=11)

# Showing Downt. Toronto as Red on the map ....
folium.CircleMarker([latitude, longitude], radius=10, popup=label, color='Red', fill=True, fill_color='Red', fill_opacity=0.9,).add_to(map_dwtor)

# ... and the Neighbourhoods as Blue.
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        ).add_to(map_dwtor)  
    
map_dwtor

## Using Foursquare API

In [30]:
CLIENT_ID = 'TKM0PEJ0HBO0P3PUQC5WHAVLKBLYHBKL1X4D1XQBJTI5P04C'      
CLIENT_SECRET = 'ONSLHRN2P1E2EWV2QWXVNPNATEY0Y2XWHU2NU2IJWMLA2H0Z'   
VERSION = '20180901' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TKM0PEJ0HBO0P3PUQC5WHAVLKBLYHBKL1X4D1XQBJTI5P04C
CLIENT_SECRET:ONSLHRN2P1E2EWV2QWXVNPNATEY0Y2XWHU2NU2IJWMLA2H0Z


In [32]:
def NearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [34]:
radius = 500
LIMIT = 100 # this will retreive data for only 100 venues per neighbourhood
Toronto_venues = NearbyVenues(names = toronto_data['Neighborhood'], latitudes = toronto_data['Latitude'], longitudes = toronto_data['Longitude'])



Central Bay Street
Studio District
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Harbourfront, Regent Park


In [35]:
print(Toronto_venues.shape)
Toronto_venues.head(10)

(195, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Central Bay Street,43.657952,-79.387383,Jimmy's Coffee,43.658421,-79.385613,Coffee Shop
1,Central Bay Street,43.657952,-79.387383,Tim Hortons,43.658570,-79.385123,Coffee Shop
2,Central Bay Street,43.657952,-79.387383,Hailed Coffee,43.658833,-79.383684,Coffee Shop
3,Central Bay Street,43.657952,-79.387383,The Elm Tree Restaurant,43.657397,-79.383761,Modern European Restaurant
4,Central Bay Street,43.657952,-79.387383,The Queen and Beaver Public House,43.657472,-79.383524,Gastropub
5,Central Bay Street,43.657952,-79.387383,Mercatto,43.660391,-79.387664,Italian Restaurant
6,Central Bay Street,43.657952,-79.387383,Panago,43.658258,-79.384313,Pizza Place
7,Central Bay Street,43.657952,-79.387383,Japango,43.655268,-79.385165,Sushi Restaurant
8,Central Bay Street,43.657952,-79.387383,Sansotei Ramen 三草亭,43.655157,-79.386501,Ramen Restaurant
9,Central Bay Street,43.657952,-79.387383,College Park Area,43.659751,-79.384911,Park


In [36]:
#pd.set_option('display.max_colwidth', 1)
Toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",16,16,16,16,16,16
Central Bay Street,88,88,88,88,88,88
"Harbourfront, Regent Park",52,52,52,52,52,52
Studio District,39,39,39,39,39,39


Lets now see how many unique venue categories are there and how rare or frequent is their occurence.


In [37]:
Toronto_venues.groupby('Venue Category').count()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Airport,1,1,1,1,1,1
Airport Food Court,1,1,1,1,1,1
Airport Gate,1,1,1,1,1,1
Airport Lounge,2,2,2,2,2,2
Airport Service,2,2,2,2,2,2
Airport Terminal,2,2,2,2,2,2
American Restaurant,3,3,3,3,3,3
Antique Shop,1,1,1,1,1,1
Art Gallery,1,1,1,1,1,1


In [38]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 94 uniques categories.


## Frequency of occurence of each category in a neighborhood

In [39]:
# one hot encoding
one_hot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")
print(one_hot.head(10))

   Airport  Airport Food Court  Airport Gate  Airport Lounge  Airport Service  \
0        0                   0             0               0                0   
1        0                   0             0               0                0   
2        0                   0             0               0                0   
3        0                   0             0               0                0   
4        0                   0             0               0                0   
5        0                   0             0               0                0   
6        0                   0             0               0                0   
7        0                   0             0               0                0   
8        0                   0             0               0                0   
9        0                   0             0               0                0   

   Airport Terminal  American Restaurant  Antique Shop  Art Gallery  \
0                 0                  

In [40]:
# add neighborhood column back to dataframe
one_hot['Neighbourhood'] = Toronto_venues['Neighbourhood'] ## I labeled the new columns as Neighbourhood with a
# 'u' bcs there is already a column named 'Neighborhood' in the venue categories.

# move neighborhood column to the first column
fixed_columns = [one_hot.columns[-1]] + list(one_hot.columns[:-1])
one_hot = one_hot[fixed_columns] 

one_hot.head(5)

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Art Museum,Asian Restaurant,Bakery,Bank,Bar,Beer Store,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Café,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Comic Shop,Convenience Store,Cosmetics Shop,Coworking Space,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fish Market,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gastropub,Gay Bar,Greek Restaurant,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Latin American Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Neighborhood,Office,Park,Performing Arts Venue,Pizza Place,Plane,Playground,Poke Place,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salad Place,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Smoothie Shop,Spa,Stationery Store,Steakhouse,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Vegetarian / Vegan Restaurant,Wine Bar,Yoga Studio
0,Central Bay Street,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Central Bay Street,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Central Bay Street,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Central Bay Street,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Central Bay Street,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [41]:
one_hot.shape

(195, 95)

## Find the mean frequency of occurence of each category in a neighborhood

In [42]:
torronto_grouped = one_hot.groupby('Neighbourhood').mean().reset_index()
torronto_grouped.head()

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Art Museum,Asian Restaurant,Bakery,Bank,Bar,Beer Store,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Café,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Comic Shop,Convenience Store,Cosmetics Shop,Coworking Space,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fish Market,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gastropub,Gay Bar,Greek Restaurant,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Latin American Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Neighborhood,Office,Park,Performing Arts Venue,Pizza Place,Plane,Playground,Poke Place,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salad Place,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Smoothie Shop,Spa,Stationery Store,Steakhouse,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Vegetarian / Vegan Restaurant,Wine Bar,Yoga Studio
0,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.0625,0.0625,0.0625,0.125,0.125,0.125,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500,0.000000,0.0625,0.000000,0.0625,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0625,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0625,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0625,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Central Bay Street,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.011364,0.000000,0.000000,0.011364,0.000000,0.011364,0.000000,0.022727,0.000000,0.0000,0.011364,0.0000,0.000000,0.000000,0.022727,0.034091,0.056818,0.000000,0.022727,0.000000,0.011364,0.147727,0.000000,0.011364,0.000000,0.000000,0.000000,0.011364,0.011364,0.011364,0.011364,0.011364,0.000000,0.000000,0.011364,0.000000,0.000000,0.011364,0.022727,0.011364,0.011364,0.000000,0.000000,0.022727,0.0000,0.000000,0.000000,0.011364,0.034091,0.011364,0.056818,0.022727,0.011364,0.011364,0.000000,0.000000,0.022727,0.011364,0.011364,0.000000,0.011364,0.011364,0.000000,0.011364,0.0000,0.000000,0.011364,0.011364,0.000000,0.011364,0.022727,0.022727,0.034091,0.0000,0.011364,0.000000,0.011364,0.022727,0.000000,0.011364,0.011364,0.011364,0.011364,0.000000,0.000000,0.011364,0.011364,0.011364
2,"Harbourfront, Regent Park",0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.019231,0.019231,0.000000,0.019231,0.057692,0.019231,0.000000,0.019231,0.0000,0.000000,0.0000,0.038462,0.019231,0.000000,0.000000,0.057692,0.000000,0.000000,0.019231,0.000000,0.153846,0.000000,0.000000,0.000000,0.019231,0.000000,0.000000,0.019231,0.000000,0.000000,0.000000,0.019231,0.019231,0.000000,0.019231,0.000000,0.019231,0.000000,0.000000,0.000000,0.000000,0.019231,0.019231,0.0000,0.019231,0.019231,0.000000,0.019231,0.000000,0.019231,0.000000,0.000000,0.000000,0.000000,0.038462,0.000000,0.000000,0.000000,0.000000,0.000000,0.057692,0.019231,0.000000,0.0000,0.019231,0.000000,0.000000,0.057692,0.000000,0.038462,0.000000,0.000000,0.0000,0.000000,0.019231,0.000000,0.019231,0.000000,0.000000,0.000000,0.000000,0.000000,0.038462,0.000000,0.000000,0.000000,0.019231
3,Studio District,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.051282,0.000

In [43]:
torronto_grouped.shape

(4, 95)

In [44]:
num_top_venues = 5

for hood in torronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = torronto_grouped[torronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
                venue  freq
0      Airport Lounge  0.12
1     Airport Service  0.12
2    Airport Terminal  0.12
3             Airport  0.06
4  Airport Food Court  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.15
1                Café  0.06
2  Italian Restaurant  0.06
3      Sandwich Place  0.03
4      Ice Cream Shop  0.03


----Harbourfront, Regent Park----
         venue  freq
0  Coffee Shop  0.15
1         Park  0.06
2         Café  0.06
3          Pub  0.06
4       Bakery  0.06


----Studio District----
                 venue  freq
0                 Café  0.10
1          Coffee Shop  0.08
2   Italian Restaurant  0.05
3  American Restaurant  0.05
4               Bakery  0.05




## Top 10 most common venues per neighborhood and put them into a dataframe

In [45]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [46]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# Labelling columns as 1st, 2nd and so on
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind])) # for 1st, 2nd, 3rd
    except:
        columns.append('{}th Most Common Venue'.format(ind+1)) ### for 4th, 5th,...

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns) ## assign column names we just created to a new dataframe
neighborhoods_venues_sorted['Neighbourhood'] = torronto_grouped['Neighbourhood']## add neighborhoods column

for ind in np.arange(torronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(torronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Service,Airport Terminal,Airport,Harbor / Marina,Plane,Boutique,Boat or Ferry,Bar,Sculpture Garden
1,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Ice Cream Shop,Burger Joint,Sandwich Place,Restaurant,Middle Eastern Restaurant,Gym / Fitness Center,Fried Chicken Joint
2,"Harbourfront, Regent Park",Coffee Shop,Park,Café,Pub,Bakery,Breakfast Spot,Theater,Mexican Restaurant,Restaurant,Ice Cream Shop
3,Studio District,Café,Coffee Shop,American Restaurant,Bakery,Italian Restaurant,Convenience Store,Middle Eastern Restaurant,Latin American Restaurant,Ice Cream Shop,Gym / Fitness Center


## Clustering the Neighborhoods

In [64]:
# set number of clusters
k = 4

torronto_grouped_clustering = torronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=k, random_state=0).fit(torronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:38] 

array([0, 3, 2, 1], dtype=int32)

In [65]:
torronto_merged = toronto_data

# add clustering labels
torronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
torronto_merged = torronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighborhood')

torronto_merged # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Italian Restaurant,Café,Ice Cream Shop,Burger Joint,Sandwich Place,Restaurant,Middle Eastern Restaurant,Gym / Fitness Center,Fried Chicken Joint
1,M4M,East Toronto,Studio District,43.659526,-79.340923,3,Café,Coffee Shop,American Restaurant,Bakery,Italian Restaurant,Convenience Store,Middle Eastern Restaurant,Latin American Restaurant,Ice Cream Shop,Gym / Fitness Center
2,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420,2,Airport Lounge,Airport Service,Airport Terminal,Airport,Harbor / Marina,Plane,Boutique,Boat or Ferry,Bar,Sculpture Garden
3,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,1,Coffee Shop,Park,Café,Pub,Bakery,Breakfast Spot,Theater,Mexican Restaurant,Restaurant,Ice Cream Shop


## Visualize the clusters

Lets now visualize the clusters using Folium library.

In [66]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k)
ys = [i+x+(i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(torronto_merged['Latitude'], torronto_merged['Longitude'], torronto_merged['Neighborhood'], torronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examining clusters

In [67]:
# Cluster 0, Red

torronto_merged.loc[torronto_merged['Cluster Labels'] == 0, torronto_merged.columns[[1] + list(range(5, torronto_merged.shape[1]))]] #red

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Italian Restaurant,Café,Ice Cream Shop,Burger Joint,Sandwich Place,Restaurant,Middle Eastern Restaurant,Gym / Fitness Center,Fried Chicken Joint


In [68]:
# Cluster 1
torronto_merged.loc[torronto_merged['Cluster Labels'] == 1, torronto_merged.columns[[1] + list(range(5, torronto_merged.shape[1]))]] # purple

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Downtown Toronto,1,Coffee Shop,Park,Café,Pub,Bakery,Breakfast Spot,Theater,Mexican Restaurant,Restaurant,Ice Cream Shop


In [69]:
#Cluster 2

torronto_merged.loc[torronto_merged['Cluster Labels'] == 2, torronto_merged.columns[[1] + list(range(5, torronto_merged.shape[1]))]] #blue

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,2,Airport Lounge,Airport Service,Airport Terminal,Airport,Harbor / Marina,Plane,Boutique,Boat or Ferry,Bar,Sculpture Garden


In [71]:
# Cluster 3
torronto_merged.loc[torronto_merged['Cluster Labels'] == 3, torronto_merged.columns[[1] + list(range(5, torronto_merged.shape[1]))]] #green

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,East Toronto,3,Café,Coffee Shop,American Restaurant,Bakery,Italian Restaurant,Convenience Store,Middle Eastern Restaurant,Latin American Restaurant,Ice Cream Shop,Gym / Fitness Center
